<a href="https://colab.research.google.com/github/laurenaxon/INFO5731_Spring2020/blob/master/INFO5731_Assignment_Three.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Three**

In this assignment, you are required to conduct information extraction, semantic analysis based on **the dataset you collected from assignment two**. You may use scipy and numpy package in this assignment.

# **Question 1: Understand N-gram**

(45 points). Write a python program to conduct N-gram analysis based on the dataset in your assignment two:

(1) Count the frequency of all the N-grams (N=3).

(2) Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.

(3) Extract all the **noun phrases** and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as all the 100 reviews (abstracts, or tweets). 


In [0]:
file = open("wuhan_tweets_Final.txt")
data = file.read()

In [13]:
# 1.1

import pandas as pd
from textblob import TextBlob
tgs=TextBlob(data).ngrams(3)

hist=dict()
for Ngram in tgs:
  Ngram=tuple(Ngram)
  hist[Ngram]=hist.get(Ngram,0)+1

pd.DataFrame.from_dict(hist, orient = 'index', columns=['Frequency'])


,Frequency
"(Date, Uncleaned, Cleaned)",1
"(Uncleaned, Cleaned, 0)",1
"(Cleaned, 0, 4/25/2020)",1
"(0, 4/25/2020, 22:29)",1
"(4/25/2020, 22:29, b'does)",1
...,...
"(wuhancoronaviru, msmfakenew, msm)",1
"(msmfakenew, msm, xe)",1
"(msm, xe, x)",1
"(http, co, keqgm)",1


In [19]:
#1.2

txt = TextBlob (data)
bgs=TextBlob(data).ngrams(2)


hist={}
for Ngram in bgs:
  Ngram=tuple(Ngram)
  hist[Ngram]=hist.get(Ngram,0)+1


bgsprob={}
for key, val in hist.items():
  bgsprob[key]=val/txt.words.count(key[0])


df = pd.DataFrame.from_dict(bgsprob, orient = 'index', columns=['Probability'])
df['Frequency']=[val for val in hist.values()]
df.head(20)

,Probability,Frequency
"(Date, Uncleaned)",1.000000,1
"(Uncleaned, Cleaned)",1.000000,1
"(Cleaned, 0)",1.000000,1
"(0, 4/25/2020)",1.000000,1
"(4/25/2020, 22:29)",0.006897,1
"(22:29, b'does)",1.000000,1
"(b'does, this)",1.000000,1
"(this, look)",0.058824,1
"(look, like)",1.000000,3
"(like, the)",0.066667,1


In [28]:
#1.3

nltk.download('brown')

txt=TextBlob(data)

hist1={}
for np in txt.noun_phrases:
  hist1[np]=hist1.get(np,0)+1

all_nphrases= [val for val in hist1.values()]
maxfrequency = max([val for val in hist1.values()])
nounphrase_prob=[]

for line in data.splitlines():
  txt=TextBlob(line)
  nounphrases= txt.noun_phrases
  hist2={}  
  for np in nounphrases:
    hist2[np]=hist2.get(np,0)+1
  for key, val in hist2.items():
    hist2[key] = val/maxfrequency
  nounphrase_prob.append(hist2)

documents=[n for n in range(1, len(re_prob)+1)]

df=pd.DataFrame (nounphrase_prob, index=documents)
df.fillna(0, inplace=True) 
df

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


,date uncleaned cleaned,covid19,wuhancoronavirus,vaccine,\nd\xe2\x80\xa6 https,//t.co/p4hnizmp8u' b look,action someon covid wuhancoronaviru ndoe access vaccin nd xe x xa http co p hnizmp u,@ cdnpoliticalsa1 @ muddywatersre,jobs in\xe2\x80\xa6 https,b cdnpoliticalsa muddywatersr,plagu mani good pay manufactur job xe x xa http co xeg olmrjm,b'good morning fellow inmates #,lifeinlockdown,# isolation #,isolationlife,# isolationcreation # isolationnation\xe2\x80\xa6 https,//t.co/3u5seetue6' b good morn fellow inmat lifeinlockdown isol isolationlif isolationcr isolationn xe x xa http co u seetu,charleshoo4,garypnabhan,charles,button di**..,sorry,u are\xe2\x80\xa6 https,b charleshoo garypnabhan,gay charl button di xf x f xa xa sorri,gay u xe x xa http co kdbekequzb,@ stevenchase #,decouplechina,\n\n # cdnpoli # bcpoli # vanre #,ccp,china,tiananmensquaremassacre\xe2\x80\xa6,//t.co/ktkewfdc58' b stevencha decouplechina gain steam n ncdnpoli bcpoli vanr ccp china tiananmensquaremassacr xe x xa http co ktkewfdc,@ stevenchase @ globeandmail,virus,ottawa,thuggish #,\n\n # cdnpoli # bcpoli #,china\xe2\x80\xa6,//t.co/sgpecv7qwv' b stevencha globeandmail viru busi,...,drtedros shut,tedrosliedpeopledied,tedrosout,tedrosresign,wuhanhealthorganisation\xe2\x80\xa6,//t.co/upjvbtzmou' b drtedro shut resign tedrosliedpeopledi tedrosout tedrosresign wuhanhealthorgani xe x xa http co upjvbtzmou,# chinazi,n't care,millions #,does,n't care at\xe2\x80\xa6 https,b chinazi care,chinaliedpeopledi mum care xe x xa http co oku izmttz,@ mithuna_khon,way street,easy way,up\xe2\x80\xa6 https,b mithuna khon agr fact gener,street easi way dress xe x xa http co siieudxqqw,b'govt shd,corona,public health emergency,order problem,persons crimina\xe2\x80\xa6 https,//t.co/g3eiircttj' b govt shd decid covid corona,public health emerg law amp order problem infect person crimina xe x xa http co g eiircttj,legallro,hmoindia,mohfw_india,policekurnool,@ kurnoolgoap @ lawinforce @,andhrapradeshcm,@ ysjagan @,ysrcparty hey,//t.co/fpzptsfjmj' b legallro hmoindia mohfw india policekurnool kurnoolgoap lawinforc andhrapradeshcm ysjagan ysrcparti hey xe x xa http co fpzptsfjmj,b'accurate af,wuhanchinavirus,msmfakenews,msm\xe2\x80\xa6,//t.co/keqgm7gyqd' b accur af china chinaliedpeopledi chineseviru wuhanchinaviru wuhancoronaviru msmfakenew msm xe x xa http co keqgm gyqd
1,0.013514,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.013514,0.013514,0.013514,0.013514,0.013514,0.013514,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013514,0.013514,0.013514,0.013514,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.

# **Question 2: Undersand TF-IDF and Document representation**

(40 points). Starting from the documents (all the reviews, or abstracts, or tweets) collected for assignment two, write a python program: 

(1) To build the **documents-terms weights (tf*idf) matrix bold text**.

(2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using **cosine similarity**.

In [30]:
#2.1

from sklearn.feature_extraction.text import TfidfVectorizer

doc=[doc for doc in data.splitlines()]
corpus = {v: k for v, k in enumerate(data.splitlines(),1)}
tfidf=TfidfVectorizer(input=doc, max_features=50, min_df=0.2, lowercase=True, analyzer='word', stop_words='english', ngram_range=(1,1))
tfs=tfidf.fit_transform(corpus.values())

features= tfidf.get_feature_names()
corpus=[n for n in corpus]

df=pd.DataFrame(tfs.T.todense(), index=features, columns=corpus)
df

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146
2020,0.0,0.233666,0.274361,0.274361,0.195243,0.274361,0.274361,0.233666,0.132906,0.377149,0.233666,0.233666,0.377149,0.233666,0.233666,0.214511,0.119693,0.233666,0.216599,0.376871,0.377149,0.274361,0.318690,0.377149,0.233666,0.274361,0.315749,0.340033,0.330010,0.330010,0.233666,0.135597,0.377149,0.274361,0.376871,0.233666,0.186669,0.233666,0.247804,0.486509,...,0.318690,0.318690,0.274361,0.274361,0.274361,0.274361,0.272120,0.274361,0.274361,0.318690,0.455431,0.233666,0.318690,0.274361,0.318690,0.274361,0.274361,0.377149,0.233666,0.233666,0.274361,0.274361,0.233666,0.186669,0.455431,0.233666,0.172187,0.233666,0.233666,0.233666,0.233666,0.377149,0.315749,0.274361,0.274361,0.274361,0.274361,0.274361,0.274361,0.233666
25,0.0,0.233666,0.274361,0.274361,0.195243,0.274361,0.274361,0.233666,0.132906,0.377149,0.233666,0.233666,0.377149,0.233666,0.233666,0.214511,0.119693,0.233666,0.433198,0.376871,0.377149,0.274361,0.318690,0.377149,0.233666,0.274361,0.631499,0.340033,0.165005,0.165005,0.233666,0.135597,0.377149,0.274361,0.376871,0.233666,0.186669,0.233666,0.495609,0.486509,...,0.318690,0.318690,0.274361,0.274361,0.274361,0.274361,0.272120,0.274361,0.274361,0.318690,0.455431,0.233666,0.318690,0.274361,0.318690,0.274361,0.274361,0.377149,0.233666,0.233666,0.274361,0.274361,0.233666,0.186669,0.455431,0.233666,0.172187,0.233666,0.233666,0.233666,0.233666,0.377149,0.631499,0.274361,0.274361,0.274361,0.274361,0.274361,0.274361,0.233666
http,0.0,0.277255,0.325541,0.325541,0.231664,0.325541,0.325541,0.277255,0.157698,0.000000,0.277255,0.277255,0.000000,0.277255,0.277255,0.509053,0.142021,0.277255,0.257004,0.447173,0.000000,0.325541,0.378139,0.000000,0.277255,0.325541,0.000000,0.000000,0.195786,0.195786,0.277255,0.160892,0.000000,0.325541,0.447173,0.277255,0.221490,0.277255,0.294031,0.000000,...,0.378139,0.378139,0.325541,0.325541,0.325541,0.325541,0.000000,0.325541,0.325541,0.378139,0.270194,0.277255,0.378139,0.325541,0.378139,0.325541,0.325541,0.000000,0.277255,0.277255,0.325541,0.325541,0.277255,0.221490,0.270194,0.277255,0.204307,0.277255,0.277255,0.277255,0.277255,0.000000,0.000000,0.325541,0.325541,0.325541,0.325541,0.325541,0.325541,0.277255
https,0.0,0.277255,0.325541,0.325541,0.231664,0.325541,0.325541,0.277255,0.157698,0.000000,0.277255,0.277255,0.000000,0.277255,0.277255,0.509053,0.142021,0.277255,0.257004,0.447173,0.000000,0.325541,0.378139,0.000000,0.277255,0.325541,0.000000,0.000000,0.195786,0.195786,0.277255,0.160892,0.000000,0.325541,0.447173,0.277255,0.221490,0.277255,0.294031,0.000000,...,0.378139,0.378139,0.325541,0.325541,0.325541,0.325541,0.000000,0.325541,0.325541,0.378139,0.270194,0.277255,0.378139,0.325541,0.378139,0.325541,0.325541,0.000000,0.277255,0.277255,0.325541,0.325541,0.277255,0.221490,0.270194,0.277255,0.204307,0.277255,0.277255,0.277255,0.277255,0.000000,0.000000,0.325541,0.325541,0.325541,0.325541,0.325541,0.325541,0.277255
wuhancoronaviru,0.0,0.391375,0.000000,0.000000,0.000000,0.000000,0.000000,0.391375,0.000000,0.631699,0.391375,0.391375,0.631699,0.391375,0.391375,0.000000,0.000000,0.391375,0.362789,0.000000,0.631699,0.000000,0.533785,0.631699,0.391375,0.000000,0.528860,0.569532,0.276373,0.276373,0.391375,0.000000,0.631699,0.000000,0.000000,0.391375,0.000000,0.391375,0.000000,0.000000,...,0.533785,0.533785,0.000000,0.000000,0.000000,0.000000,0.455784,0.000000,0.000000,0.533785,0.000000,0.391375,0.533785,0.000000,0.533785,0.000000,0.000000,0.631699,0.391375,0.391375,0.000000,0.000000,0.391375,0.000000,0.000000,0.391375,0.288402,0.391375,0.391375,0.391375,0.391375,0.631699,0.528860,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.391375
wuhancoronavirus,0.0,0.348539,0.000000,0.000000,0.000000,0.000000,0.000000,0.348539,0.198244,0

In [32]:
#2.2 

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
nltk.download('stopwords')
nltk.download('wordnet')

import re

from textblob import Word



query="Where did the pandemic originate?"


query=query.lower()
query=re.sub('[^\w\s]', '', query)
stop=stopwords.words('english')
query= " ".join(x for x in query.split() if x not in stop)
st=PorterStemmer()
query=" ".join([st.stem(word) for word in query.split()])
query= " ".join([Word(word).lemmatize() for word in query.split()])

print(query)


with open("wuhan_tweet_query.txt", "a", encoding='utf-8') as file:
  file.write(data)
  file.write(query)
  
with open("wuhan_tweet_query.txt", "r", encoding='utf-8') as file:
  data_query = file.read()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
pandem origin


# **Question 3: Create your own training and evaluation data for sentiment analysis**

(15 points). **You dodn't need to write program for this question!** Read each review (abstract or tweet) you collected in detail, and annotate each review with a sentiment (positive, negative, or neutral). Save the annotated dataset into a csv file with three columns (first column: document_id, clean_text, sentiment), upload the csv file to GitHub and submit the file link blew. This datset will be used for assignment four: sentiment analysis and text classification. 


In [0]:
# Main Link: 

https://github.com/laurenaxon/haihua_INFO5731_Spring2020/blob/master/wuhan_tweets_sentiments.csv



# Raw Link: 

https://raw.githubusercontent.com/laurenaxon/haihua_INFO5731_Spring2020/master/wuhan_tweets_sentiments.csv